In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('hotel_bookings.csv')
df.head() # print top 5 rows only

In [ ]:
# (#rows, #columns)
df.shape

In [ ]:
# display max # of columns
pd.set_option("display.max_columns", 32)

In [ ]:
df.head() # print top 5 rows

In [ ]:
# shows all the columns' name this data frame has
df.columns

In [ ]:
df.nunique()

<span style="color:Gray">
adr: avg daily revenue that a hotel receives for each occupied guest room by date. By measuring adr, we are able to see the avg rate that comes from all occupied rooms.<br/>
reservation_status: canceled/checked out/not show <br/>
reservation_status_date: when was the reservation_status last updated
</span>

In [ ]:
# count the number of occurrences each value have
df['hotel'].value_counts()

<span style="color:Gray">
The city hotel is having 79330 records and the resort hotel is having 40060 records. The city hotel has almost double number of records more than the resort hotel. This dataset is imbalanced.<br/>
(# important to know when creating the models)
</span>

In [ ]:
df['meal'].value_counts()

<span style="color:Gray">
"meal" has five categories<br/>
&nbsp;&nbsp;<span style="color:blue">("Undefined" is the same as "SC" -> can merge this two columns by rename "Undefined" as "SC")</span> <br/>
Board: what is included in the price of accommodation<br/>
BB: bedding Breakfast<br/>
HB: Half board, breakfast + one other meal (usually dinner)<br/>
SC: self-catering (no meals are included)<br/>
Undefined: no meal package<br/>
FB: full board, breakfast + lunch + dinner
</span>

In [ ]:
df['market_segment'].value_counts()

Online TA: online travel agent\
Offline TA/TO: offline travel agent/ tour operator

In [ ]:
df['distribution_channel'].value_counts()

In [ ]:
df['deposit_type'].value_counts()

In [ ]:
df['customer_type'].value_counts()

In [ ]:
df['reservation_status'].value_counts()

In [ ]:
df['total_of_special_requests'].value_counts()

In [ ]:
df['is_repeated_guest'].value_counts()

In [ ]:
sns.countplot(data=df, x="hotel")

In [ ]:
sns.countplot(data=df, x="is_canceled", hue="is_repeated_guest")

In [ ]:
sns.countplot(data=df, x="hotel", hue = "is_canceled")

<span style="color:Gray">
City hotel has more guests and higher percentage of cancelation compared to the resort hotel.
    </span>

# 3. Cleaning (Prepare data for analysis)

## Missing Data

In [ ]:
# check if there are any NULL values present in the df
df.isnull().values.any()

<span style="color:Gray">
We get TRUE, which means there are NULL values present in the df
    </span>

In [ ]:
# check which columns are having NULL values
df.isnull().sum()

<span style="color:Gray">
"Country" is having 488 NULL values.<br/>
"Agent" is having 16340 NULL values: 16340 orders are booked directly.</span>

<span style="color:blue">(we can replace all the NULL values with "0")</span>.

In [ ]:
# replacing missing value with 0
# inplace=True: doing it directly in df
df.fillna(0, inplace=True)

In [ ]:
df.isnull().sum()

<span style="color:Gray">
There is no more NULL values after replacing all of them with 0
    </span>

<span style="color:Red; font-size:14pt">
PS: whatever we do will not affect on the main data set. It only happens in this dataframe.
    </span>

In [ ]:
# replace "Undefined" in "Meal" with "SC"
df['meal'].replace("Undefined", "SC", inplace=True)

In [ ]:
df['meal'].unique()

In [ ]:
df['meal'].value_counts()

<span style="color:Gray">
    Now, there are 10650+1169 "SC" values in meal
    </span>

<span style="color:Gray">
    For a booking information entry, if adult = children = babies = 0, it's not a valid entry
</span>

In [ ]:
# select all the valid entries and store in "Subset"
Subset=df[ (df['adults']==0) & (df['children']==0) & (df['babies']==0)]
Subset

In [ ]:
Subset[['adults', 'babies', 'children']]

<span style="color:Gray">
    If we check the 'adults', 'children' and 'babies' columns of Subset, we can see all their values are zero.<br/>
    There are 180 invalid entries and we will delete them.
    </span>

In [ ]:
type(Subset)

In [ ]:
"""
we want to delete the entries where adults=children=babies=0
check if each entry if adults=children=babies=0; True -> delete, False -> keep
if one of ABC != 0 -> the whole expression will be False -> we do not delete this entry
"""
False_list = (df['children']==0) & (df['adults']==0) & (df['babies']==0)

In [ ]:
type(False_list)

In [ ]:
False_list

In [ ]:
# delete the negation of False_list (Series), which means deleting the entries which children = adults = babies = 0
valid = df[~False_list]

In [ ]:
valid.head() # the top five rows

In [ ]:
# check again if new data frame still contains invalid entries
temp = valid[ (valid['adults']==0) & (valid['children']==0) & (valid['babies']==0) ]
temp

In [ ]:
valid.shape
# In[4] we have (119390, 32), we removed 180 rows

<span style="color: Gray">
In[4] we have (119390, 32), we removed 180 rows, 119390-180=119210
    </span>

In [ ]:
valid.to_csv('Updataed_Hotel_Booking.csv', index=False)

# 4. Analysis (asking and answering questions)

### 1. Where do our guests com from (which country)? 

<span style="color: Gray">
We don't want to do marketing in the wrong region. We want to do marketing at the region where most of our customers come from. <br/>
    To visualize the region, we can choose bar chart, pie chart or map.
    </span>

In [ ]:
# we don't have the dataframe containing information we require
"""
get the entries where the booking is not canceled
-> get the country column
-> count the number of entries for each country
-> reset index of df (the old index is added as new column, and a new sequential index is used)
-> 
"""
# 
guest_country = valid[valid['is_canceled']==0]['country'].value_counts().reset_index()
guest_country.columns = ['country', 'Number of guests']
guest_country

we can see from the map the majority of the customers are coming from European

In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# use Bar() to create bar chart
trace = go.Bar(
    x=guest_country['country'],
    y=guest_country['Number of guests'],
    marker=dict(color="#CD7F32")
)
trace
type(trace)

In [ ]:
layout = go.Layout(title="Guests by Country") # t
fig = go.Figure(data=[trace], layout=layout)
fig

In [ ]:
pyo.plot(fig)

In [ ]:
map_guest = px.choropleth(
    guest_country, # data frame you want to cre
    locations = guest_country['country'],
    color = guest_country["Number of guests"],
    hover_name = guest_country['country'],
    title = "Home country of guests")
map_guest.show()

In [ ]:
total_guests = guest_country["Number of guests"].sum()
Guests_in_percent = round(guest_country["Number of guests"]/total_guests * 100, 2)
guest_country['Guests_in_%'] = Guests_in_percent
guest_country

In [ ]:
# use Bar() to create bar chart
pct_bar = go.Bar(
    x=guest_country['country'],
    y=guest_country['Guests_in_%'],
    marker=dict(color="#CD7F32")
)
pct_bar
type(pct_bar)
layout = go.Layout(title="%Guests by Country") # t
fig = go.Figure(data=[pct_bar], layout=layout)
fig

In [ ]:
percentage_map = px.choropleth(guest_country,
                              locations = guest_country['country'],
                              color = guest_country['Guests_in_%'],
                              hover_name = guest_country["Guests_in_%"],
                              title = "Percentage of guests' home countries")
percentage_map.show()

# 5. Room Price Analysis

Price variation per person per night over the year \
we will use average to analyze\
hotels: Resort, City\
Steps\
1: Create DF for resort and city respectively\
2. calculate average value for adr(average daily rate) in 2 dataframe\
for each type of hotel: for each month: there will be many entries -> for each date, there will be a adr (average daily value) -> the average of them will be the monthly adr\
3. combine two dataframes into one\
4. rearrange the month\
5. plot the chart

## Miss Interpreting Data

In [ ]:
valid

In [ ]:
# get the booking entries that supposed to be counted
resort = valid[(valid["is_canceled"] == 0) & (valid['hotel'] == "Resort Hotel")]
city = valid[(valid['is_canceled'] == 0) & (valid['hotel'] == "City Hotel")]
resort

In [ ]:
"""
Resort hotel's mean adr for each month
group the data by arrival_date_month --> calculate the mean adr
"""
resort_hotel = resort.groupby(['arrival_date_month'])['adr'].mean().reset_index()
resort_hotel

In [ ]:
# City hotel's mean adr for each month
city_hotel = city.groupby(["arrival_date_month"])["adr"].mean().reset_index()
city_hotel

In [ ]:
# merge the two data frame above into one by their month (SQL: a join b on xxx)
final = resort_hotel.merge(city_hotel, on="arrival_date_month")
# rename the column names
final.columns = ["month", "price_for_resort_hotel", "price_for_city_hotel"]
final

In [ ]:
# install the packages that helps sorting the month
!pip install sort-dataframeby-monthorweek
## Dependency package
!pip install sorted-months-weekdays

In [ ]:
# sort the month by ascending order(method 1)
import sort_dataframeby_monthorweek as sd
final2 = sd.Sort_Dataframeby_Month(df=final, monthcolumnname="month")
final2

In [ ]:
# method 2: specify the categories, pandas remembers the order of specification as the default sort order.
months = ["January", "February", "March", "April", "May", "June", "July", "August", 
          "September", "October", "November", "December"]
final['month'] = pd.Categorical(final['month'], categories=months, ordered=True)
final.sort_values(by='month',inplace=True)
final

In [ ]:
# create two-line chart with x as month, and y as mean adr for each month for two different type of hotel
px.line(final,
       x = "month",
       y = ["price_for_resort_hotel", "price_for_city_hotel"],
       title = "Room Price Per Night Over The Months")

<span style="color:Gray">
    The price for resort hotel is much higher during the summer (June, July and August) than other time in the year. The price for city hotel is about the same for the whole year, in the range of \$80 to \$120.<br/>
    But we are not considering the number of guests right now, which we should.
    </span>

In [ ]:
# add another column "adr_updated" which contains the adr value per person
valid["adr_updated"] = valid["adr"] / (valid["adults"] + valid["children"])
valid

In [ ]:
resort_udp = valid[ (valid["hotel"] == "Resort Hotel") & (valid['is_canceled'] == 0) ]
resort_hotel_udp = resort_udp.groupby( ['arrival_date_month'] )['adr_updated'].mean().reset_index()
resort_hotel_udp

In [ ]:
city_udp = valid[ (valid["hotel"] == "City Hotel") & (valid['is_canceled'] == 0) ]
city_hotel_udp = city_udp.groupby( ["arrival_date_month"] )['adr_updated'].mean().reset_index()
city_hotel_udp

In [ ]:
final_udp = resort_hotel_udp.merge( city_hotel_udp, on="arrival_date_month")
final_udp.columns = ['month', 'price_for_resort_hotel', 'price_for_city_hotel']
final_udp

In [ ]:
final_udp = sd.Sort_Dataframeby_Month(df = final_udp, monthcolumnname="month")
final_udp

In [ ]:
# create two-line chart with x as month, and y as mean adr for each month for two different type of hotel
px.line(final_udp,
       x = "month",
       y = ["price_for_resort_hotel", "price_for_city_hotel"],
       title = "Room Price Per Night Per Person Over The Months")

Summer in Portugal sits between June and mid-September<br/>
while the winter season falls between December
can't say whether the price is increasing or decreasing since it depends on many factors

# 6. Room Price Analysis Across Room Type
How is the price distributed across the room type? <br/>
To answer this question, we will need column **hotel, assigned_room_type** and **adr** <br/>
<span style="color:Gray">
Box plot: <br/>
x: room type <br/>
y: price (assume price in EU)<br/>
one line for each room type
    </span>

In [ ]:
valid.columns

In [ ]:
# check how many different types of room are there in the hotel
df['reserved_room_type'].unique()

In [ ]:
"""
loc函数：通过行索引 "Index" 中的具体值来取行数据（如取"Index"为"A"的行）

iloc函数：通过行号来取行数据（如取第二行的数据）
"""
valid_guest = valid.loc[ valid["is_canceled"] == 0]
prices = valid_guest[ ["hotel", "reserved_room_type", "adr_updated"] ].sort_values("reserved_room_type")
prices

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x="reserved_room_type",
           y="adr_updated", 
           hue = "hotel",
           data=prices)

plt.title("Price of room types per night and person", fontsize = 16)
plt.xlabel("Room type", fontsize=16)
plt.ylabel("Price [EUR]", fontsize=16)
plt.ylim(0,160)   # show the price range in (0, 160)
plt.show()

In [ ]:
# see price for type C room
prices_C = prices[ prices["reserved_room_type"] == "C" ]
prices_C

In [ ]:
# prices of type C romm for both city and resort hotel
prices_C_city = prices_C[ prices_C['hotel']=='City Hotel' ]
prices_C_resort = prices_C[ prices_C['hotel']=="Resort Hotel" ]
prices_C_city

In [ ]:
prices_C_city.describe()

In [ ]:
prices_C_resort.describe()

# 7. How Long Do Guest Stay

In [ ]:
# we have stays_in_weekend_nights and stays_in_week_nights
not_canceled = valid[valid['is_canceled']== 0]  # 0: not canceled, 1: canceled
not_canceled

In [ ]:
"""
Adding new column to DF using loc[]

not_canceled.loc[:,"total_nights"] = not_canceled["stays_in_weekend_nights"] + not_canceled["stays_in_week_nights"]
not_canceleds
"""

In [ ]:
not_canceled["total_nights"] = not_canceled["stays_in_weekend_nights"] + not_canceled["stays_in_week_nights"]
not_canceled

In [ ]:
# extract the three columns we want to use
not_canceled_temp = not_canceled[["hotel", "is_canceled", "total_nights"]]
not_canceled_temp

In [ ]:
#hotel_stay = not_canceled_temp.groupby(["hotel", "total_nights"]).agg("count").reset_index()
hotel_stay = not_canceled_temp.groupby(["total_nights", "hotel"]).agg("count").reset_index()
hotel_stay

In [ ]:
"""
is_canceled" is counting how many "not canceled" are there, 
which is number of orders staying for total of xx nights. For readability, it will be renamed as "Number of stays"
"""
hotel_stay = hotel_stay.rename( columns= {'is_canceled' : "Number of stays"} )
hotel_stay.head()

In [ ]:
hotel_stay_resort = hotel_stay[ hotel_stay['hotel'] == 'Resort Hotel' ]
hotel_stay_resort

In [ ]:
hotel_stay_city = hotel_stay[ hotel_stay['hotel'] == "City Hotel" ]
hotel_stay_city

In [ ]:

trace_resort = go.Bar(
    x = hotel_stay_resort["total_nights"],
    y = hotel_stay_resort["Number of stays"],
    name = "Resort Stay"
)
trace_city = go.Bar(
    x = hotel_stay_city["total_nights"],
    y = hotel_stay_city["Number of stays"],
    name = "City Stay"
)

trace_list = [trace_resort, trace_city]
layout = go.Layout(title = "Total Number of stays by Guest")
fig = go.Figure(data=trace_list, layout=layout)
# pyo.plot(fig) #output htmls
fig

In [ ]:
total_stay = hotel_stay_resort["Number of stays"].sum() + hotel_stay_city["Number of stays"].sum()
hotel_stay_resort["percentage stay"] = hotel_stay_resort["Number of stays"]/total_stay * 100
hotel_stay_resort

In [ ]:
hotel_stay_city["percentage stay"] = hotel_stay_city["Number of stays"]/total_stay * 100
hotel_stay_city

In [ ]:
[hotel_stay_resort["percentage stay"].sum(), hotel_stay_city["percentage stay"].sum()]

Most (77%) of the customers are staying for one to four night. If people are staying for 1 to 5 nights, they are more likely to choose city hotel, otherwise they are more likely to choose resort hotel.

# 8. Booking By Market segment

### 8.1 Which market segment has higher bookings?

In [ ]:
segments = valid["market_segment"].value_counts()
segments

In [ ]:
# make a pie chart for market segments
fig = px.pie(segments, 
            values = segments.values,
            names = segments.index, 
            title = "Bookings per market segment",
            template = "seaborn")
fig.update_traces(rotation = -90, textinfo = "percent+label")
fig.show()

67.6% booking are booked through travel agent, and 47.3% of those are through online TA and 20.3% are through offline TA.

### 8.2 What is the price distribution through market segment?

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(x = "market_segment", 
            y = "adr_updated",
            hue = "reserved_room_type", # adding second layer of grouping within each x values
            data = valid,
            ci=None # no error line
           )
plt.title("ADR by market segment and room type", fontsize = 16)
plt.xlabel("Market segment", fontsize=16)
plt.xticks(rotation=45) # rotate x label
plt.ylabel("ADR per person [EUR]", fontsize = 16)
plt.legend(loc = "upper left")
plt.show()

Room type E with booking through aviation has the highest average daily rate per person.\
Room type L only present in direct booking. We might say room type L only offer to people booking directly.

In [ ]:
# What is the price distribution for all room type?

# 9. Booking Cacellation

### 9.1 How many bookings were cancelled?

In [ ]:
cancel = valid["is_canceled"] == 1
cancel

In [ ]:
resort_cancellation = valid.loc[valid["hotel"] == "Resort Hotel"]["is_canceled"].sum()
city_cancellation = valid.loc[valid["hotel"] == "City Hotel"]["is_canceled"].sum()
resort_cancellation, city_cancellation

In [ ]:
new_line = '\n'
print(f"Total Booking Cancelled: {cancel.sum()}. {new_line}Total Resort Hotel Booking Cancelled: {resort_cancellation}. {new_line}Total Resort Hotel Booking Cancelled: {city_cancellation}.")

### 9.2 Which month have the highest number of cancelations?

In [ ]:
resort_groupby_month = valid.loc[ valid["hotel"]=="Resort Hotel" ].groupby("arrival_date_month")
resort_book_per_month = resort_groupby_month["hotel"].count()
resort_cancel_per_month = resort_groupby_month["is_canceled"].sum() # is_canceled == 1 == yes, count() is counting total number of entries -> use sum() instead of count() 
resort_cancel_per_month

In [ ]:
city_groupby_month = valid.loc[ valid["hotel"]=="City Hotel" ].groupby("arrival_date_month")
city_book_per_month = city_groupby_month["hotel"].count()
city_cancel_per_month = city_groupby_month["is_canceled"].sum() 
city_cancel_per_month

In [ ]:
res_cancel_data = pd.DataFrame({"Hotel": "Resort Hotel",
                               "Month": resort_book_per_month.index,
                               "Bookings": resort_book_per_month.values,
                               "Cancelations": resort_cancel_per_month.values})
res_cancel_data = sd.Sort_Dataframeby_Month(df=res_cancel_data, monthcolumnname="Month")
city_cancel_data = pd.DataFrame({"Hotel": "City Hotel",
                                "Month": city_book_per_month.index,
                                "Bookings": city_book_per_month.values,
                                "Cancelations": city_cancel_per_month.values})
city_cancel_data = sd.Sort_Dataframeby_Month(df = city_cancel_data, monthcolumnname = "Month")
res_cancel_data

In [ ]:
plt.figure(figsize = (12,8))

trace_res = go.Bar(
    x = res_cancel_data["Month"],
    y = res_cancel_data["Cancelations"],
    name = "Rst Cancelled"
)
trace_city = go.Bar(
    x = city_cancel_data['Month'],
    y = city_cancel_data["Cancelations"],
    name = "Cty Cancelled ")
layout = go.Layout(
    title = "Total Number of Cancelations by Guest"
)
fig = go.Figure(data=[trace_res,trace_city], layout=layout)
fig

May has the highest number of cancelations of 3653 for city hotel, and August has the highest number of cancelations of 1637 for resort hotel.